In [1]:
import numpy as np
import torch
from torch import nn, optim, autograd
from torch.nn import functional as F
from pyDOE import lhs
import scipy.io
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable

from models_all import *

#Paper reproduction
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)

In [2]:
N_train = 10000


# x,t
la = np.array([1,1])
lb = np.array([-1,0])

traindata = lb+(la-lb)*lhs(2,N_train)


x = traindata[:,0:1]
t = traindata[:,1:2]

x = torch.from_numpy(x).float()
t = torch.from_numpy(t).float()

x.requires_grad_()
t.requires_grad_()

###########GPU###########
x = x.cuda()
t = t.cuda()
###########GPU###########

In [3]:
def func(x):
    return np.sin(np.pi * x[:, 0:1]) * np.exp(-x[:, 1:])

In [4]:
observe_number = 200

observe_data = lb+(la-lb)*lhs(2,observe_number)
observe_y = func(observe_data)
observe_data = torch.from_numpy(observe_data).float()
observe_y = torch.from_numpy(observe_y).float()

###########GPU###########
observe_y = observe_y.cuda()
observe_data = observe_data.cuda()
###########GPU###########

In [5]:
N_test = 2000
test_data = lb+(la-lb)*lhs(2,N_test)
test_y = func(test_data)
test_data = torch.from_numpy(test_data).float()
test_y = torch.from_numpy(test_y).float()
###########GPU###########
test_data = test_data.cuda()
test_y = test_y.cuda()
###########GPU###########

In [6]:
def relative_l2(u_pred, u_real):

    l2 = torch.norm(u_real - u_pred, p=2) / torch.norm(u_real, p=2)
    
    return l2.item()  # Convert the result back to a Python float

In [7]:
C1 = torch.tensor(2.0, requires_grad=True)

In [8]:
def output_transform(x, y):
    x_in = x[:, 0:1]
    t_in = x[:, 1:2]

    return (1 - x_in) * (1 + x_in) * (1 - torch.exp(-t_in)) * y + torch.sin(np.pi * x_in)

In [9]:
def Loss_f(x_inside,y_inside,PINNs,C,return_sequence='not'):
    
    ########### loss f  ###########
    E_inside = PINNs(torch.cat((x_inside,y_inside),axis=1))
    E_inside = output_transform(torch.cat((x_inside,y_inside),axis=1), E_inside)
    
    d_t = autograd.grad(outputs=E_inside, inputs=y_inside,
                              grad_outputs=torch.ones_like(E_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]  

    d_x = autograd.grad(outputs=E_inside, inputs=x_inside,
                              grad_outputs=torch.ones_like(E_inside),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]  

    d_xx = autograd.grad(outputs=d_x, inputs=x_inside,
                              grad_outputs=torch.ones_like(d_x),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]  


    loss_f_sequence = d_t-C*d_xx+torch.exp(-y_inside)*(torch.sin(torch.tensor(np.pi)*x_inside)-torch.tensor(np.pi)*torch.tensor(np.pi)*torch.sin(torch.tensor(np.pi)*x_inside))
    loss_f_sequence = torch.square(loss_f_sequence)
    loss_f = torch.mean(loss_f_sequence)

    
    
    #loss_f = torch.mean(torch.square(E_t-C1*(E_xx)-5*(E_inside-E_inside**3)))
    ########### loss f  ###########
    
    if return_sequence=='yes':
        return loss_f_sequence
    else:
        return loss_f

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
    print("Using CUDA on GPU")
else:
    device = "cpu"
    print("Using CPU")

In [ ]:
traindata = np.concatenate((np.linspace(-1,1,200).reshape(-1,1),np.linspace(0,1,200).reshape(-1,1)),1)
x_ = traindata[:,0:1]
y_ = traindata[:,1:2]
xx,yy = np.meshgrid(x_,y_)
data_numpy = np.concatenate((xx.reshape(-1,1),yy.reshape(-1,1)),1)
plot_data_tensor = torch.from_numpy(data_numpy).float()
plot_data_tensor = plot_data_tensor.cuda()

aa = func(data_numpy)
plt.imshow(aa.reshape(200,200),extent=[-1, 1, 0, 1], cmap='rainbow')
plt.colorbar(shrink=.5)
#plt.scatter(gan_data_x_t[:,0:1], gan_data_x_t[:,1:2],c='k',marker = 'x',alpha=1,s=15)
#plt.savefig('Possion h points.eps',format='eps',dpi=1000, bbox_inches = 'tight')

plt.show()

## PINN

In [ ]:
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
np.random.seed(1234)


# ##############PW_L_f###############
# e_k = 0.01
# q_k = 0.0001
# s_k = 0.5
# Ada_CL_L_f = Ada_CL(e_k,q_k,s_k,device)
# weight_sequence_L_f = (1/N_train)*np.ones((N_train,1))
# ##############PW_L_f###############    

##############PW_L_f###############
e_k = 0.005
q_k = 0.005
s_k = 0.3


Ada_CL_L_T = Ada_CL(e_k,q_k,s_k,device)
weight_sequence_L_T = (1/observe_number)*np.ones((observe_number,1))
##############PW_L_f###############    


PINNs1 = NN_H2(2, 32, 6, 1)
PINNs1.cuda()

# PINNs1.apply(weights_init)

import torch.nn.init as init
for name, param in PINNs1.named_parameters():
    if 'weight' in name:
        init.xavier_uniform_(param)


optimizer1 = optim.Adam(PINNs1.parameters(), lr=0.001,betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

optimizer1.add_param_group({'params': [C1], 'lr': 0.001})

nIter1 = 10000


loss_all_1 = []
loss_f_1 = []
loss_all_2 = []
loss_T_1 = []
test_loss_1 = []
C1_list = []

loss_f_sequence_list = []
weight_sequence_L_f_list = []
plot_heat_list = []

loss1_value = 1
it = 0


while  it<6000:

    
    ##### loss f  ######    
#     loss_f_sequence = Loss_f(x,t,PINNs1,C1,return_sequence='yes')
#     ##############PW_L_f###############
#     loss_f,weight_sequence_L_f = Ada_CL_L_f.Training_Scheduler_torch(loss_f_sequence, weight_sequence_L_f)
    loss_f = Loss_f(x,t,PINNs1,C1,return_sequence='not')  
    ##############PW_L_f###############    

    
    ###### loss observation  ######        
    E_observation = PINNs1(observe_data)
    E_observation = output_transform(observe_data, E_observation)   
    loss_observation,weight_sequence_L_T = Ada_CL_L_T.Training_Scheduler_torch(torch.square(E_observation-observe_y), weight_sequence_L_T)
  
    ####### loss PI #######
    loss = loss_f+10*loss_observation
    
    
    #########  test_loss NRMSE  #########
    pre_y = PINNs1(test_data)
    pre_y = output_transform(test_data, pre_y)     
    test_loss = relative_l2(pre_y,test_y)
    #########  test_loss NRMSE  #########   
    
    ############## Record ###############
    loss_f_1.append(loss_f.item())
    test_loss_1.append(test_loss)
    C1_list.append(C1.item())   
    loss_T_1.append(loss_observation.item()) 
    loss_all_1.append(loss.item())
    ############## Record ###############
  
    ############## Record Original ###############
    loss_observation_original = Loss_f(x,t,PINNs1,C1,return_sequence='not') 
    loss_original = loss_f+10*loss_observation_original 
    loss_all_2.append(loss_original.item())
    ############## Record Original ###############
    

    if it % 1000 == 0:
        loss_f_sequence_list.append(loss_observation.cpu().detach().numpy())
        weight_sequence_L_f_list.append(weight_sequence_L_T)
        pre_u_for_plot = PINNs1(plot_data_tensor)
        pre_u_for_plot = output_transform(plot_data_tensor, pre_u_for_plot)     
        plot_heat_list.append(pre_u_for_plot.cpu().detach().numpy())
        print('It:', it, 'train_loss:', loss.item(), 'test_loss:', test_loss) 
    
    optimizer1.zero_grad()
    loss.backward()
    optimizer1.step()
    
        
    it = it + 1     
    
print('Final:', 'train_loss:', loss.item(), 'test_loss:', test_loss) 